In [1]:
import requests
import logging
import pandas as pd

In [2]:
r = requests.get('https://rickandmortyapi.com/api/location/?page=1')

In [3]:
r.json()['info']

{'count': 126,
 'pages': 7,
 'next': 'https://rickandmortyapi.com/api/location/?page=2',
 'prev': None}

In [4]:
r.json()['results'][0]

{'id': 1,
 'name': 'Earth (C-137)',
 'type': 'Planet',
 'dimension': 'Dimension C-137',
 'residents': ['https://rickandmortyapi.com/api/character/38',
  'https://rickandmortyapi.com/api/character/45',
  'https://rickandmortyapi.com/api/character/71',
  'https://rickandmortyapi.com/api/character/82',
  'https://rickandmortyapi.com/api/character/83',
  'https://rickandmortyapi.com/api/character/92',
  'https://rickandmortyapi.com/api/character/112',
  'https://rickandmortyapi.com/api/character/114',
  'https://rickandmortyapi.com/api/character/116',
  'https://rickandmortyapi.com/api/character/117',
  'https://rickandmortyapi.com/api/character/120',
  'https://rickandmortyapi.com/api/character/127',
  'https://rickandmortyapi.com/api/character/155',
  'https://rickandmortyapi.com/api/character/169',
  'https://rickandmortyapi.com/api/character/175',
  'https://rickandmortyapi.com/api/character/179',
  'https://rickandmortyapi.com/api/character/186',
  'https://rickandmortyapi.com/api/cha

In [5]:
r.json()['info']['pages']

7

In [6]:
def get_page_count(ram_api_url):
    '''
    Get page count in RAM
    :param ram_api_url
    :return ram_page_count
    '''
    r = requests.get(ram_api_url)
    if r.status_code == 200:
        ram_page_count = r.json()['info']['pages']
        logging.info('SUCCESS')
        logging.info(f'ram_page_count = {ram_page_count}')
        return ram_page_count
    else:
        logging.warning(f'HTTP STATUS {r.status_code}')
        # raise AirflowException('Error in load page count')

In [7]:
get_page_count('https://rickandmortyapi.com/api/location/?page=1')

7

In [8]:
def get_location_info(result_json):
    lst = list()
    for lctn in result_json:
        lst.append((lctn['id'], lctn['name'], lctn['type'], lctn['dimension'], len(lctn['residents'])))
    return lst

In [9]:
get_location_info(r.json()['results'])

[(1, 'Earth (C-137)', 'Planet', 'Dimension C-137', 27),
 (2, 'Abadango', 'Cluster', 'unknown', 1),
 (3, 'Citadel of Ricks', 'Space station', 'unknown', 101),
 (4, "Worldender's lair", 'Planet', 'unknown', 9),
 (5, 'Anatomy Park', 'Microverse', 'Dimension C-137', 11),
 (6, 'Interdimensional Cable', 'TV', 'unknown', 62),
 (7, 'Immortality Field Resort', 'Resort', 'unknown', 3),
 (8, 'Post-Apocalyptic Earth', 'Planet', 'Post-Apocalyptic Dimension', 10),
 (9, 'Purge Planet', 'Planet', 'Replacement Dimension', 4),
 (10, 'Venzenulon 7', 'Planet', 'unknown', 1),
 (11, 'Bepis 9', 'Planet', 'unknown', 1),
 (12, 'Cronenberg Earth', 'Planet', 'Cronenberg Dimension', 0),
 (13, 'Nuptia 4', 'Planet', 'unknown', 11),
 (14, "Giant's Town", 'Fantasy town', 'Fantasy Dimension', 3),
 (15, 'Bird World', 'Planet', 'unknown', 0),
 (16, 'St. Gloopy Noops Hospital', 'Space station', 'unknown', 9),
 (17, 'Earth (5-126)', 'Planet', 'Dimension 5-126', 0),
 (18, "Mr. Goldenfold's dream", 'Dream', 'Dimension C-137

In [10]:
first_page_url = 'https://rickandmortyapi.com/api/location/?page=1'

In [11]:
lctns_lst = list()
for pg in range(1, get_page_count(first_page_url) + 1):
    rqst = requests.get(f'https://rickandmortyapi.com/api/location/?page={pg}')
    if rqst.status_code == 200:
        lctns_lst.extend(get_location_info(rqst.json()['results']))
    else:
        print('failed')

In [12]:
df = pd.DataFrame(data=lctns_lst, columns=['id', 'name', 'type', 'dimension', 'resident_cnt']).sort_values(by='resident_cnt', ascending=False).reset_index(drop=True).iloc[0:3,:]
df

,id,name,type,dimension,resident_cnt
0,20,Earth (Replacement Dimension),Planet,Replacement Dimension,230
1,3,Citadel of Ricks,Space station,unknown,101
2,6,Interdimensional Cable,TV,unknown,62


In [13]:
list(df.itertuples(index=False, name=None))

[(20, 'Earth (Replacement Dimension)', 'Planet', 'Replacement Dimension', 230),
 (3, 'Citadel of Ricks', 'Space station', 'unknown', 101),
 (6, 'Interdimensional Cable', 'TV', 'unknown', 62)]